In [2]:
#Data Preprocessing
import scipy.io
import numpy as np

def scaler():
    data = scipy.io.loadmat('data_for_cnn.mat')
    X = data['ecg_in_window']
    data = scipy.io.loadmat('class_label.mat')
    Y = data['label']
    valueArray = np.concatenate((X, Y), axis = 1)
    np.random.shuffle(valueArray)
    X = valueArray[0:, :1000]
    Y = valueArray[0:, -1]
    Y = np.reshape(Y,(Y.shape[0],1))
    Y = Y.astype(int)
    #Feature Scaling
    X = (X - np.mean(X, axis = 0))/np.std(X, axis = 0)
    X = X.reshape(X.shape[0], X.shape[1], 1)
    #Output in correct format
    Y = np.eye(len(np.unique(Y,axis=0)))[(Y.T).flatten()]
    return X, Y

In [3]:
X, Y = scaler()

In [4]:
# Holdout Method
train_size = 0.8
train_X = X[0: int(train_size * X.shape[0])]
test_X = X[int(train_size * X.shape[0]):]

In [19]:
train_X.shape, test_X.shape

((800, 1000, 1), (200, 1000, 1))

In [5]:
from keras.models import Model, Sequential
from keras.layers import Dense, Conv1D, Flatten, Lambda, MaxPooling1D, UpSampling1D, Conv2DTranspose, Input,Reshape
from keras.engine.topology import Layer
import keras
import keras.backend as K

Using TensorFlow backend.


In [6]:
class Conv1DTranspose(Layer):
    def __init__(self, filters, kernel_size, strides=1, *args, **kwargs):
        self._filters = filters
        self._kernel_size = (1, kernel_size)
        self._strides = (1, strides)
        self._args, self._kwargs = args, kwargs
        super(Conv1DTranspose, self).__init__()

    def build(self, input_shape):
        print("build", input_shape)
        self._model = Sequential()
        self._model.add(Lambda(lambda x: K.expand_dims(x,axis=1), batch_input_shape=input_shape))
        self._model.add(Conv2DTranspose(self._filters,
                                        kernel_size=self._kernel_size,
                                        strides=self._strides,
                                        *self._args, **self._kwargs))
        self._model.add(Lambda(lambda x: x[:,0]))
        self._model.summary()
        super(Conv1DTranspose, self).build(input_shape)

    def call(self, x):
        return self._model(x)

    def compute_output_shape(self, input_shape):
        return self._model.compute_output_shape(input_shape)

In [ ]:
filters = 64

# Encoder
input_data = Input(shape = (1000,1))
l1 = Conv1D(filters, 10, strides = 2, activation = 'relu')(input_data)
l2 = MaxPooling1D(2)(l1)
l3 = Flatten()(l2)
l4 = Dense(248 * filters, activation = 'relu')(l3)
# Decoder
l4 = Reshape((248, filters))(l4)
l5 = UpSampling1D(2)(l4)
output_data = Conv1DTranspose(1, 10, strides = 2)(l5)
model = Model(input_data, output_data)
model.compile(loss = 'mean_squared_error', optimizer = 'adam')
model.fit(train_X, train_X, validation_data = (test_X, test_X), batch_size = 100, epochs = 50)

build (None, 496, 64)
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_8 (Lambda)            (None, 1, 496, 64)        0         
_________________________________________________________________
conv2d_transpose_5 (Conv2DTr (None, 1, 1000, 1)        641       
_________________________________________________________________
lambda_9 (Lambda)            (None, 1000, 1)           0         
Total params: 641
Trainable params: 641
Non-trainable params: 0
_________________________________________________________________
